In [1]:
import random

"""
1.Data generator
    a. Loads vocab
    b. Loads image features
    c. provide data for training
2. Builds image caption model.
3. Trains the model.
"""
import os
import sys
import pprint
import tensorflow.compat.v1 as tf
import tensorflow.io.gfile as gfile
from tensorflow.compat.v1 import logging
import pprint
import _pickle as cPickle
import numpy as np
import math

tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.INFO)

input_description_file = "./image_caption_data/results_20130124.token"
input_img_feature_file = "./image_caption_data/feature_extraction_inception_v3"
input_vocab_file = "./image_caption_data/vocab.txt"
output_dir = "./image_caption_data/local_run"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

class HParams:
    def __init__(self,
                 num_vocab_word_threshold,
                 num_embedding_nodes,
                 num_timesteps,
                 num_lstm_nodes,
                 num_lstm_layers,
                 num_fc_nodes,
                 batch_size,
                 cell_type,
                 clip_lstm_grades,
                 learning_rate,
                 keep_prob,
                 log_frequent,
                 save_frequent):
        self.num_vocab_word_threshold = num_vocab_word_threshold
        self.num_embedding_nodes = num_embedding_nodes
        self.num_timesteps = num_timesteps
        self.num_lstm_nodes = num_lstm_nodes
        self.num_lstm_layers = num_lstm_layers
        self.num_fc_nodes = num_fc_nodes
        self.batch_size = batch_size
        self.cell_type = cell_type
        self.clip_lstm_grades = clip_lstm_grades
        self.learning_rate = learning_rate
        self.keep_prob = keep_prob
        self.log_frequent = log_frequent
        self.save_frequent = save_frequent

def get_default_params():
    return HParams(
        num_vocab_word_threshold = 3,
        num_embedding_nodes = 32,
        num_timesteps = 10,
        num_lstm_nodes = [64,64],
        num_lstm_layers = 2,
        num_fc_nodes = 32,
        batch_size = 80,
        cell_type = "lstm",
        clip_lstm_grades = 1.0,
        learning_rate = 0.001,
        keep_prob = 0.8,
        log_frequent = 10,
        save_frequent = 100
    )

hps = get_default_params()
print(hps.save_frequent)

Instructions for updating:
non-resource variables are not supported in the long term
100


In [2]:
class Vocab(object):
    def __init__(self, filename, word_num_threshold):
        self._id_to_word = {}
        self._word_to_id = {}
        self._unk = -1
        self._eos = -1
        self._word_num_threshold = word_num_threshold
        self._read_dict(filename)

    def _read_dict(self, filename):
        with gfile.GFile(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            word, occurrence = line.strip('\r\n').split('\t')
            occurrence = int(occurrence)
            if occurrence < self._word_num_threshold:
                continue
            idx = len(self._id_to_word)
            if word == '<UNK>':
                self._unk = idx
            elif word == '.':
                self._eos = idx
            if word in self._word_to_id or idx in self._id_to_word:
                raise Exception("duplicate words in vocab.")
            self._word_to_id[word] = idx
            self._id_to_word[idx] = word

    @property
    def unk(self):
        return self._unk
    @property
    def eos(self):
        return self._eos
    def word_to_id(self, word):
        return self._word_to_id.get(word, self.unk)
    def id_to_word(self, word):
        return self._id_to_word.get(word, '<UNK>')
    def size(self):
        return len(self._id_to_word)
    def encode(self, sentence):
        return [self.word_to_id(word) for word in sentence.split(' ')]
    def decode(self, sentence_id):
        words = [self.id_to_word(word_id) for word_id in sentence_id]
        return ' '.join(words)

In [3]:
vocab = Vocab(input_vocab_file, hps.num_vocab_word_threshold)
vocab_size = vocab.size()
logging.info("vocab_size: %d" % vocab_size)
pprint.pprint(vocab.encode("I have a dream."))
pprint.pprint(vocab.decode([5, 10, 9, 20]))

INFO:tensorflow:vocab_size: 10875
[1494, 389, 1, 0]
'the of man white'


In [4]:
def parse_token_file(token_file):
    """Parses image description file."""
    img_name_to_tokens = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            img_id, description = line.strip('\r\n').split('\t')
            img_name, _ = img_id.split('#')
            img_name_to_tokens.setdefault(img_name, [])
            img_name_to_tokens[img_name].append(description)
    return img_name_to_tokens

def convert_token_to_id(img_name_to_tokens, vocab):
    """Converts tokens of each description of imgs to id."""
    img_name_to_tokens_id = {}
    for img_name in img_name_to_tokens:
        img_name_to_tokens_id.setdefault(img_name, [])
        for description in img_name_to_tokens[img_name]:
            token_ids = vocab.encode(description)
            img_name_to_tokens_id[img_name].append(token_ids)
    return img_name_to_tokens_id

In [5]:
img_name_to_tokens = parse_token_file(input_description_file)
img_name_to_tokens_id = convert_token_to_id(img_name_to_tokens, vocab)


In [6]:
logging.info("num of all imgs: %d" % len(img_name_to_tokens))
pprint.pprint(img_name_to_tokens['2778832101.jpg'])
logging.info("num if all imgs: %d" % len(img_name_to_tokens_id))
pprint.pprint(img_name_to_tokens_id['2778832101.jpg'])


INFO:tensorflow:num of all imgs: 31783
['A man in jeans is reclining on a green metal bench along a busy sidewalk and '
 'crowded street .',
 'A white male with a blue sweater and gray pants laying on a sidewalk bench .',
 'A man in a blue shirt and gray pants is sleeping on a sidewalk bench .',
 'A person is sleeping on a bench , next to cars .',
 'A man sleeping on a bench in a city area .']
INFO:tensorflow:num if all imgs: 31783
[[3, 9, 4, 132, 8, 3532, 6, 1, 48, 337, 146, 139, 1, 244, 93, 7, 380, 36, 2],
 [3, 20, 179, 11, 1, 26, 284, 7, 120, 128, 297, 6, 1, 93, 146, 2],
 [3, 9, 4, 1, 26, 21, 7, 120, 128, 8, 340, 6, 1, 93, 146, 2],
 [3, 63, 8, 340, 6, 1, 146, 12, 70, 15, 518, 2],
 [3, 9, 340, 6, 1, 146, 4, 1, 112, 171, 2]]


In [7]:
class ImageCaptionData(object):
    """Provides data for image caption model."""
    def __init__(self,
                 image_name_to_token_id,
                 image_feature_dir,
                 num_timesteps,
                 vocab,
                 deterministic = False):
        self._vocab = vocab
        self._image_name_to_token_id = image_name_to_token_id
        self._num_timesteps = num_timesteps
        self._deterministic = deterministic
        self._indicator = 0

        self._img_feature_filenames = []
        self._img_feature_data = []

        self._all_img_feature_filepaths = []
        for filename in gfile.listdir(image_feature_dir):
            self._all_img_feature_filepaths.append(
                os.path.join(image_feature_dir,filename)
            )
        pprint.pprint(self._all_img_feature_filepaths)
        self._load_img_feature_pickle()

        if not self._deterministic:
            self._random_shuffle()

    def _load_img_feature_pickle(self):
        """Load img feature data from pickle"""
        for filepath in self._all_img_feature_filepaths:
            logging.info("loading %s" % filepath)
            with gfile.GFile(filepath, 'rb') as  f:
                filenames, features = cPickle.load(f,encoding = 'iso-8859-1')
                self._img_feature_filenames += filenames
                self._img_feature_data.append(features)
        # 按照最后一个维度合并
        self._img_feature_data = np.vstack(self._img_feature_data)
        origin_shape = self._img_feature_data.shape
        # 去掉中间两个维度
        self._img_feature_data = np.reshape(
            self._img_feature_data,
            (origin_shape[0], origin_shape[3])
        )
        self._img_feature_filenames = np.asarray(self._img_feature_filenames)
        print(self._img_feature_data.shape)
        print(self._img_feature_filenames.shape)

    def size(self):
        return len(self._img_feature_filenames)

    def img_feature_size(self):
        return self._img_feature_data.shape[1]

    def _random_shuffle(self):
        """Shuffle data randomly."""
        p = np.random.permutation(self.size())
        self._img_feature_filenames = self._img_feature_filenames[p]
        self._img_feature_data = self._img_feature_data[p]

    def img_desc(self, batch_filenames):
        """Gets descriptions for filenames in batch."""
        batch_sentence_ids = []
        batch_wights = []
        for filename in batch_filenames:
            token_ids_set = self._image_name_to_token_id[filename]
            chosen_token_ids = random.choice(token_ids_set)
            chosen_token_ids_length = len(chosen_token_ids)

            weight = [1 for i in range(chosen_token_ids_length)]
            if chosen_token_ids_length >= self._num_timesteps:
                chosen_token_ids = chosen_token_ids[0:self._num_timesteps]
                weight = weight[0:self._num_timesteps]
            else:
                remaining_length = self._num_timesteps - chosen_token_ids_length
                chosen_token_ids += [self._vocab.eos for i in range(remaining_length)]
                weight += [0 for i in range(remaining_length)]
            batch_sentence_ids.append(chosen_token_ids)
            batch_wights.append(weight)
        batch_sentence_ids = np.asarray(batch_sentence_ids)
        batch_wights = np.asarray(batch_wights)
        return batch_sentence_ids, batch_wights

    def next_batch(self, batch_size):
        """Returns next batch data."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self.size():
            if not self._deterministic:
                self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self.size()

        batch_filenames = self._img_feature_filenames[self._indicator: end_indicator]
        batch_img_features = self._img_feature_data[self._indicator: end_indicator]
        # sentence_ids: [100,101,0,0,0] -> [1,1,0,0,0]
        # batch_weights: 计算权重,不计算<UNK>的损失函数
        batch_sentence_ids, batch_weights = self.img_desc(batch_filenames)
        self._indicator = end_indicator
        return batch_img_features, batch_sentence_ids, batch_weights, batch_filenames

In [8]:
caption_data = ImageCaptionData(img_name_to_tokens_id,
                                input_img_feature_file,
                                hps.num_timesteps,
                                vocab)

img_feature_dim = caption_data.img_feature_size()
caption_data_size = caption_data.size()
logging.info("img_features_dim: %d" % img_feature_dim)
logging.info("caption_data_size: %d" % caption_data_size)

batch_img_features, batch_sentence_ids, batch_weights, batch_img_names = caption_data.next_batch(5)
pprint.pprint(batch_img_features)
pprint.pprint(batch_sentence_ids)
pprint.pprint(batch_weights)
pprint.pprint(batch_img_names)

['./image_caption_data/feature_extraction_inception_v3\\image_features-0.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-1.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-10.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-11.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-12.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-13.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-14.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-15.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-16.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-17.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-18.pickle',
 './image_caption_data/feature_extraction_inception_v3\\image_features-19.pickle',
 './im

In [9]:
def create_rnn_cell(hidden_dim, cell_type):
    """Returns specific cell according to cell_type"""
    if cell_type == 'lstm':
        return tf.nn.rnn_cell.BasicLSTMCell(hidden_dim,
                                            state_is_tuple = True)
    elif cell_type == 'gru':
        return tf.nn.rnn_cell.GRUCell(hidden_dim)
    else:
        raise Exception("%s type has not been supported." % cell_type)

def dropout(cell, keep_prob):
    """Wrap cell with dropout."""
    return tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob = keep_prob)

def get_train_model(hps, vocab_size, img_feature_dim):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size

    img_feature = tf.placeholder(tf.float32,
                                 (batch_size, img_feature_dim))
    sentence = tf.placeholder(tf.int32,
                              (batch_size, num_timesteps))

    mask = tf.placeholder(tf.int32,
                          (batch_size, num_timesteps))
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
    global_step = tf.Variable(tf.zeros([], tf.int32),
                              name = "global_step",
                              trainable = False)
    # prediction process:
    # sentence: [a, b, c, d, e]
    # input: [image, a, b, c, d]
    # image_feature: [0.4, 0.3, 10, 2]
    # predict #1: image_feature -> embedding_img -> lstm -> (a)
    # predict #2: a -> embedding_word -> lstm -> (b)
    # predict #3: b -> embedding_word -> lstm -> (c)
    # ....

    # Sets up embedding layer.
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embeddings',
                           initializer = embedding_initializer):
        embeddings = tf.get_variable(
            'embedding',
            [vocab_size, hps.num_embedding_nodes],
            tf.float32)
        # embed_token_ids: [batch_size, num_timesteps - 1, num_embedding_nodes]
        embed_token_ids = tf.nn.embedding_lookup(
            embeddings,
            sentence[:,0: num_timesteps - 1])
        img_feature_embed_init = tf.uniform_unit_scaling_initializer(
            factor = 1.0)
        with tf.variable_scope('img_feature_embed',
                               initializer = img_feature_embed_init):
            # img_feature: [batch_size, img_feature_dim]
            # embed_img: [batch_size, num_embedding_nodes]
            embed_img = tf.layers.dense(img_feature, hps.num_embedding_nodes)
            # embed_img: [batch_size, 1, num_embedding_nodes]
            embed_img = tf.expand_dims(embed_img, 1)
            #embed_inputs: [batch_size, num_timesteps, num_embedding_nodes]
            embed_inputs = tf.concat([embed_img, embed_token_ids], axis = 1)

        # Sets up rnn network
        scale = 1.0 / math.sqrt(hps.num_embedding_nodes + hps.num_lstm_nodes[-1]) / 3.0
        rnn_init = tf.random_uniform_initializer(-scale, scale)
        with tf.variable_scope('lstm_nn', initializer = rnn_init):
            cells = []
            for i in range(hps.num_lstm_layers):
                cell = create_rnn_cell(hps.num_lstm_nodes[i], hps.cell_type)
                cell = dropout(cell, keep_prob)
                cells.append(cell)
            cell = tf.nn.rnn_cell.MultiRNNCell(cells)
            init_state = cell.zero_state(hps.batch_size, tf.float32)
            # rnn_outputs: [batch_size, num_timestep, hps.num_lstm_node[-1]]
            rnn_outputs, _ = tf.nn.dynamic_rnn(cell,
                                               embed_inputs,
                                               initial_state = init_state)
        # Sets up fully-connected layer
        fc_init = tf.uniform_unit_scaling_initializer(factor = 1.0)
        with tf.variable_scope('fc', initializer = fc_init):
            rnn_outputs_2d = tf.reshape(rnn_outputs,
                                        [-1, hps.num_lstm_nodes[-1]])
            fc1  = tf.layers.dense(rnn_outputs_2d,
                                  hps.num_fc_nodes,
                                  name = "fc1")
            fc1_dropout = tf.layers.dropout(fc1, keep_prob)
            fc1_relu = tf.nn.relu(fc1_dropout)
            logits = tf.layers.dense(fc1_relu,
                                     vocab_size,
                                     name = "logits")

        # Calculates loss
        with tf.variable_scope('loss'):
            sentence_flatten = tf.reshape(sentence, [-1])
            mask_flatten = tf.reshape(mask, [-1])
            mask_sum = tf.reduce_sum(mask_flatten)
            mask_flatten = tf.cast(mask_flatten, tf.float32)
            mask_sum = tf.cast(mask_sum, tf.float32)

            softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = logits,
                labels = sentence_flatten)

            weighted_softmax_loss = tf.multiply(
                softmax_loss, tf.cast(mask_flatten, tf.float32))
            loss = tf.reduce_sum(weighted_softmax_loss) / tf.cast(mask_sum, tf.float32)

            prediction = tf.argmax(logits, 1, output_type = tf.int32)
            correct_prediction = tf.equal(prediction,
                                          sentence_flatten)
            weighted_correct_prediction = tf.multiply(
                tf.cast(correct_prediction, tf.float32),
                tf.cast(mask_flatten, tf.float32))
            print(prediction)
            # predict_sentence = [vocab.id_to_word(each_sentence) for each_sentence in sentence]
            # if global_step % 100 == 0:
            #     print(predict_sentence)
            accuracy = tf.reduce_sum(weighted_correct_prediction) / mask_sum
            tf.summary.scalar('loss', loss)

        # Defines train op.
        with tf.variable_scope('train_op'):
            tvars = tf.trainable_variables()
            for var in tvars:
                logging.info('variable name: %s' % var.name)
                grads, _ = tf.clip_by_global_norm(
                    tf.gradients(loss, tvars), hps.clip_lstm_grades)
                optimizer = tf.train.AdamOptimizer(hps.learning_rate)
                train_op = optimizer.apply_gradients(
                    zip(grads, tvars),global_step = global_step)

        return ((img_feature, sentence, mask, keep_prob),
                (loss, accuracy, train_op),
                global_step,prediction)

place_holders, metrics, global_step, prediction = get_train_model(hps, vocab_size, img_feature_dim)
img_feature, sentence, mask, keep_prob = place_holders
loss, accuracy, train_op = metrics

summary_op = tf.summary.merge_all()
init_op = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep = 10)


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.
Tensor("embeddings/loss/ArgMax:0", shape=(800,), dtype=int32)
INFO:tensorflow:variable name: embeddings/embedding:0
INFO:tensorflow

In [10]:
training_steps = 10000
with tf.Session() as sess:
    sess.run(init_op)
    writer = tf.summary.FileWriter(output_dir, sess.graph)
    for i in range(training_steps):
        (batch_img_features,
         batch_sentence_ids,
         batch_weights, _) = caption_data.next_batch(hps.batch_size)
        input_vals = (batch_img_features,
                      batch_sentence_ids,
                      batch_weights,
                      hps.keep_prob)
        feed_dict = dict(zip(place_holders, input_vals))
        fetches = [global_step, loss, accuracy, prediction, train_op]
        should_log = (i + 1) % hps.log_frequent == 0
        should_save = (i + 1) % hps.save_frequent == 0
        if should_log:
            fetches += [summary_op]

        outputs = sess.run(fetches, feed_dict = feed_dict)
        global_step_val, loss_val, accuracy_val, prediction_ = outputs[0:4]
        if should_log:
            summary_str = outputs[-1]
            writer.add_summary(summary_str, global_step_val)
            logging.info('Step: %5d, loss: %3.3f, accu: %3.3f'
                         % (global_step_val, loss_val, accuracy_val))

        if should_save:
            model_save_file = os.path.join(output_dir, "image_caption")
            logging.info('Step: %5d, model saved' % global_step_val)
            saver.save(sess, model_save_file, global_step = global_step_val)
            predict_sentence = [vocab.id_to_word(word) for word in prediction_]
            predict_sentence_list = [predict_sentence[i:i + 10] for i in
                                     range(0, len(predict_sentence), 10)]
            for i in range(9):
                print(predict_sentence[i],end = " ")
            print(predict_sentence[10])

INFO:tensorflow:Step:    10, loss: 9.247, accu: 0.001
INFO:tensorflow:Step:    20, loss: 8.957, accu: 0.000
INFO:tensorflow:Step:    30, loss: 8.502, accu: 0.073
INFO:tensorflow:Step:    40, loss: 8.027, accu: 0.059
INFO:tensorflow:Step:    50, loss: 7.227, accu: 0.075
INFO:tensorflow:Step:    60, loss: 6.424, accu: 0.111
INFO:tensorflow:Step:    70, loss: 6.090, accu: 0.150
INFO:tensorflow:Step:    80, loss: 5.837, accu: 0.143
INFO:tensorflow:Step:    90, loss: 5.644, accu: 0.163
INFO:tensorflow:Step:   100, loss: 5.795, accu: 0.147
INFO:tensorflow:Step:   100, model saved
A man a a a a a a a A
INFO:tensorflow:Step:   110, loss: 5.805, accu: 0.131
INFO:tensorflow:Step:   120, loss: 5.639, accu: 0.149
INFO:tensorflow:Step:   130, loss: 5.434, accu: 0.164
INFO:tensorflow:Step:   140, loss: 5.323, accu: 0.171
INFO:tensorflow:Step:   150, loss: 5.555, accu: 0.167
INFO:tensorflow:Step:   160, loss: 5.375, accu: 0.173
INFO:tensorflow:Step:   170, loss: 5.539, accu: 0.162
INFO:tensorflow:Ste